# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [ ]:
! pip install -U langchain

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)



## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [1]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [21]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""
# 生物、计算机和汉语文学老师
biology_template = """你是一位杰出的生物学家，擅长用简单明了的方式解释复杂的生物学问题。
你善于用生动的例子和类比来帮助理解。
如果你不确定某个问题的答案，你会坦诚承认，并愿意进一步查找信息。

这是一个问题：
{input}"""
computer_template = """你是一位非常厉害的计算机专家。
你擅长回答计算机科学的问题，尤其是编程和算法。
你能够将复杂的问题分解成简单的部分，逐步解决，并用易懂的方式解释。

这是一个问题：
{input}"""
chinese_template = """你是一位非常博学的汉语言文学老师。
你擅长用通俗易懂的方式解释汉语言文学的问题。
当你不知道某个问题的答案时，你会坦诚承认，并且愿意进一步查找资料。

这是一个问题：
{input}"""

In [22]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
     {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biology_template,
    },
     {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": computer_template,
    },
     {
        "name": "汉语言",
        "description": "适用于回答汉语言文学问题",
        "prompt_template": chinese_template,
    }
]

In [23]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", base_url="https://api.xiaoai.plus/v1", api_key="sk-YIOnMFnuWMDaLG0bC93a20F2456b4dAdAa4aE692449483E7")

In [24]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [25]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [26]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [27]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [28]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机问题', '汉语言: 适用于回答汉语言文学问题']


In [10]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言: 适用于回答汉语言文学问题


In [30]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [31]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言: 适用于回答汉语言文学问题

<< INPUT >>
{input}

<< 

In [32]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [34]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？?'}
> Finished chain.
{'input': '黑体辐射是什么？?', 'text': '\n\n黑体辐射是指一种理想化的物理理论模型，它假设一个能够吸收和辐射所有电磁辐射的物体。根据这个理论，黑体在所有温度下都会辐射出特定的能量，其辐射出的光谱分布被称为黑体辐射谱。黑体辐射是研究光谱学和热力学的重要基础，也被应用于太阳能电池、红外线热成像等领域。'}


In [15]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '40'}
> Finished chain.
{'input': '40', 'text': '+60+50\n\n这个问题可以分解为三部分：40+60、60+50、40+50。\n根据加法的交换律，这三部分的结果都是150。\n因此，40+60+50的答案为150。'}


In [17]:
router_chain.verbose = True

In [18]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '黑洞是什么？'}
> Finished chain.
{'input': '黑洞是什么？', 'text': '\n\n黑洞是一种极端密度和强大引力的天体，它的重力场非常强大，甚至连光也无法逃脱。它的存在是由于恒星在死亡时发生了坍缩，使得它们的质量集中在极小的空间内。黑洞的存在已经被多项观测和理论证据所证实，但仍然有很多未解之谜。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [29]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言: 适用于回答汉语言文学问题


In [35]:
print(chain.invoke("细胞分裂和细胞分化有什么关系?"))



> Entering new MultiPromptChain chain...
生物: {'input': '细胞分裂和细胞分化有什么关系?'}
> Finished chain.
{'input': '细胞分裂和细胞分化有什么关系?', 'text': '\n\n细胞分裂和细胞分化是两个不同的生物学过程，但它们之间有密切的关系。\n\n细胞分裂是指一个细胞分裂成两个或更多细胞的过程，这是生物体生长和发育的基础。在细胞分裂过程中，细胞的遗传物质DNA会复制，然后平均分配给每个新细胞，确保每个细胞都具有相同的遗传信息。\n\n细胞分化是指细胞在发育过程中发生的变化，使其成为不同类型的细胞。这是一个非常复杂的过程，通常涉及到细胞内部基因的调节和表达。分化细胞会具有特定的结构和功能，以适应其所处的组织和器官的需要'}


In [36]:
print(chain.invoke("MapReduce和Spark在处理数据上有什么差别?"))



> Entering new MultiPromptChain chain...
计算机: {'input': 'MapReduce和Spark在处理数据上有什么差别?'}
> Finished chain.
{'input': 'MapReduce和Spark在处理数据上有什么差别?', 'text': '\n\nMapReduce和Spark都是用于分布式计算的框架，都可以用来处理大规模的数据。它们的主要差别在于它们的架构和运行效率。\n\nMapReduce是一种较早的分布式计算框架，它由Google开发，并在Apache Hadoop中得到了广泛应用。它的核心思想是将大规模的数据分成小块，然后通过map和reduce操作来处理这些数据。在map操作中，数据被分割成多个子集，然后在不同的计算节点上进行处理。在reduce操作中，这些子集的结果会被合并为最终的结果。MapReduce的优势是简单易用，并且可以运行在廉价的硬件上。但是它也有一些缺点，比如需要多次的磁盘读写操作，导致效率较低。\n\nSpark是近年来发展起来'}


In [37]:
print(chain.invoke("中文有几个音调?"))



> Entering new MultiPromptChain chain...
汉语言: {'input': '汉语言有几个声调?'}
> Finished chain.
{'input': '汉语言有几个声调?', 'text': '\n\n汉语言有四个声调。'}


In [38]:
print(chain.invoke("红楼梦里面为什么孙二娘要杀武松?"))



> Entering new MultiPromptChain chain...
汉语言: {'input': '红楼梦里面为什么孙二娘要杀武松?'}
> Finished chain.
{'input': '红楼梦里面为什么孙二娘要杀武松?', 'text': '\n\n孙二娘要杀武松的原因有多重，主要有以下几点：\n1. 嫉妒：孙二娘嫉妒武松娶了潘金莲，而自己却被武大抛弃，心生怨恨。\n2. 欲望：孙二娘私藏了武松的一封信件，希望能够与武松发生关系，但被武松发现并拒绝，从而激起了她的欲望和报复心理。\n3. 避免丑闻：孙二娘担心武松会泄露她的秘密，为了避免丑闻，决定杀武松。\n4. 堆积的压力：孙二娘的丈夫被武松'}


In [39]:
print(chain.invoke("红楼梦不是没有武松吗?"))



> Entering new MultiPromptChain chain...
汉语言: {'input': '红楼梦不是没有武松吗?'}
> Finished chain.
{'input': '红楼梦不是没有武松吗?', 'text': '\n\n不是的，红楼梦中的武松是《水浒传》中的人物，在《红楼梦》中并没有出现。可能是因为《水浒传》和《红楼梦》都是中国四大经典小说之一，所以有些人会混淆两部作品中的人物。'}
